References:

1. IMP-PCMLAI-M22-customising-CNN_solution-Copy1.ipynb

Rework this, like a sketch, based on the solution and then splice back in.

It might be easier to go the other way, but let's see.

Minimialist as possible, but with good comments/ annotations.

# Question 1 solution:
## Introduction
...

In [1]:
# GB: Packages:
# GB: From previous file:
import torch # GB: Contains torch.utils.data?
import torchvision
import torchvision.transforms as transforms
# GB: Added:
import cv2 # For img_loader.
import numpy as np
from torchvision.datasets import ImageFolder # GB: For Helper Functions.
# GB: Ref.: https://pytorch.org/vision/main/generated/torchvision.datasets.ImageFolder.html
# GB: Ref.: https://discuss.pytorch.org/t/how-to-load-images-from-different-folders-in-the-same-batch/18942/6
print(torch.__version__) # GB: Added.

import torch.nn as nn
import torch.nn.functional as F

# GB: 3.
import torch.optim as optim
import time

# GB: 4.
from hyperopt import fmin, hp, tpe # GB: In alphabetical order.
# GB: Rationalise at the end.

2.1.0.post100


In [2]:
# GB: Suffix Structure:
# GB: _train
# GB: _validate # GB: NB Not used.
# GB: _test

## Data

In [3]:
# GB: Helper Functions:
def img_loader(filename):
    return cv2.imread(filename, cv2.IMREAD_GRAYSCALE)

def img_folder(root):
    # GB: Changed from 3D to 1D, i.e. from (0.5, 0.5, 0.5) to (0.5,). NB The trailing comma might be important.
    # GB: Parameters: means and standard deviations.
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    return ImageFolder(root=root, loader=img_loader, transform=transform)

# GB: Datasets:
folder_train = './grayscale/train/' # GB: Put in here to not get lost.
folder_test = './grayscale/test/' # GB: Put in here to not get lost.
dataset_train = img_folder(folder_train)
dataset_test = img_folder(folder_test)

# GB: Check:
array_train = np.array(dataset_train.classes)
array_test = np.array(dataset_test.classes)

if np.array_equal(array_train, array_test):
    print('Classes are equal', dataset_train.classes)
    classes = dataset_train.classes
else:
    print('Classes are not equal', dataset_train.classes, dataset_test.classes)
    classes = ['error']

Classes are equal ['bart_simpson', 'charles_montgomery_burns', 'homer_simpson', 'krusty_the_clown', 'lisa_simpson', 'marge_simpson', 'milhouse_van_houten', 'moe_szyslak', 'ned_flanders', 'principal_skinner']


In [4]:
# GB: Dataloaders:
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=20, shuffle=True, num_workers=0)
dataloader_test  = torch.utils.data.DataLoader(dataset_test,  batch_size=20, shuffle=True, num_workers=0)

# GB: Check:
# GB: Okay.

## Modelling 

In [5]:
# GB: Model:

# GB: Parameters (like space below):
thisdict = {
  'brand': "Ford",
  'model': "Mustang",
  'year': 1964
}



# GB: Parameters:
dim_1 = 29 * 27
dim_2 = 13 * 12
dim_4 = 5 * 4
dim_ = dim_1 # GB: Default: dim_4.

c_1 = 20
c_2 = 4
c_3 = 2
c_ = c_2 # GB: Default: ?
# Add Switch for # GB: Could do as adapted or original LeNet?

print(thisdict["brand"])

# GB: Start Boilerplate (PyTorch):
class Net(nn.Module):
    def __init__(self, name=None):
        # GB: For what each layer does see ...
        # GB: Ref.: https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html
        super(Net, self).__init__()
        if name:
            self.name = name
        self.conv1 = nn.Conv2d(1, 6, 5) # self.conv1 = nn.Conv2d(1, 10, 5)
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)) # GB: vs 2, 2. # self.pool = nn.MaxPool2d((3, 2), 2)
        self.conv2 = nn.Conv2d(6, 16, 5) # self.conv2 = nn.Conv2d(10, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120) # self.fc1 = nn.Linear(192, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
        # compute the total number of parameters
        total_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        print(self.name + ': total params:', total_params)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4) # x = x.view(-1, 192)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net(name='GavNet-1') # GB: Changed from net to model for clarity.
print(model) # GB: Added.
# GB: End Boilerplate (PyTorch).

Ford
GavNet-1: total params: 44426
Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


# Training and testing

In [6]:
# %%
# GB: Packages.3.

# GB: Train Function:
def train(params, *args, **kwargs):
    # GB: dataloader_train and model previously declared.
    # GB: Hyperparameters:
    lr, momentum, epochs = params['lr'], params['momentum'], params['epochs']
    lr = 0.001 # GB: Manual overide.
    momentum = 0.9 # GB: Manual overide.
    epochs = 2 # GB: Manual overide.
    # GB: Parameters:
    

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum) # GB: Changed from net to model.
    # GB: Ref.: https://pytorch.org/docs/stable/generated/torch.optim.SGD.html

    start = time.time()

    for epoch in range(epochs):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(dataloader_train, 0):
            # get the inputs; data is a list of [inputs, labels] where from?
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics # GB: Adjust for expected data.
            running_loss += loss.item()
            if i % 20 == 19:    # print every 2000 mini-batches # GB: Needs tweeked for dataset. # GB: Chnaged from 2000, 1999
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 20)) # GB: Chnaged from 20.
                running_loss = 0.0

    end = time.time()
    print('training time ', end - start)
    print('Finished Training')

In [7]:
def test(params, *args, **kwargs):
    # GB: classes and dataloader_test and model previously declared.
    # GB: Hyperparameters:
    # GB: Spare.
    correct = 0
    total = 0
    with torch.no_grad():
        tmp = 0 # GB: Temporary metric.
        for data in dataloader_test:
            tmp += 1 
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('runs of batches', tmp)
    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))

    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    with torch.no_grad():
        for data in dataloader_test:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    for i in range(10):
        print('Accuracy of %5s : %2d %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))

    return round(1 - (correct / total), 6) # GB: Six decimal places aligned to Capstone project.

## Hyperopt

In [12]:
# GB: At this point it had finished running. Make replacement for main(())
# GB: Inspired by [2]
# GB: https://github.com/hyperopt/hyperopt/wiki/FMin
# GB: https://github.com/hyperopt/hyperopt/wiki/FMin#11-the-simplest-case
# GB: Better ref than previous line
# GB: https://hyperopt.github.io/hyperopt/tutorials/02.MultipleParameterTutorial/

# GB: Trying... $ conda install intel::hyperopt # GB: Failed.
# GB: Trying... $ pip install hyperopt # GB: Passed with errors.
# GB: Come back to.

# GB: Packages.4

# Helper Function:
def run_model(params):
    train(params)
    test_result = test(params)
    return test_result

# GB: Hyperparameters:
space = {
    'lr': hp.uniform('lr', 0.05, 0.1), # GB: Defaults: 0.001(PyTorch)-?.
    'momentum': hp.uniform('momentum', 0.1, 0.5), # GB: Defaults: 0(PyTorch)-0.9.
    'epochs': hp.uniformint('epochs', 1, 5) # GB: Defaults: ?-5(PyTorch).
}

best = fmin(fn=run_model,
            space=space,
            algo=tpe.suggest,
            max_evals=3) # GB: Defaults: 2(test)-50.

print(best)

[1,    20] loss: 0.001                                                          
[1,    40] loss: 0.001                                                          
[1,    60] loss: 0.001                                                          
[1,    80] loss: 0.001                                                          
[1,   100] loss: 0.001                                                          
[1,   120] loss: 0.001                                                          
[1,   140] loss: 0.001                                                          
[1,   160] loss: 0.001                                                          
[1,   180] loss: 0.001                                                          
[1,   200] loss: 0.001                                                          
[1,   220] loss: 0.001                                                          
[1,   240] loss: 0.001                                                          
[1,   260] loss: 0.001      

Accuracy of the network on the 10000 test images: 56 %                          
Accuracy of bart_simpson : 36 %                                                 
Accuracy of charles_montgomery_burns : 58 %                                     
Accuracy of homer_simpson : 42 %                                                
Accuracy of krusty_the_clown : 54 %                                             
Accuracy of lisa_simpson : 60 %                                                 
Accuracy of marge_simpson : 59 %                                                
Accuracy of milhouse_van_houten : 64 %                                          
Accuracy of moe_szyslak : 54 %                                                  
Accuracy of ned_flanders : 42 %                                                 
Accuracy of principal_skinner : 61 %                                            
[1,    20] loss: 0.001                                                          
[1,    40] loss: 0.001      

[2,   300] loss: 0.001                                                          
[2,   320] loss: 0.001                                                          
[2,   340] loss: 0.001                                                          
[2,   360] loss: 0.001                                                          
[2,   380] loss: 0.001                                                          
[2,   400] loss: 0.001                                                          
training time                                                                   
5.442919969558716                                                               
Finished Training                                                               
runs of batches                                                                 
100                                                                             
Accuracy of the network on the 10000 test images: 55 %                          
Accuracy of bart_simpson : 5

[2,    80] loss: 0.001                                                          
[2,   100] loss: 0.001                                                          
[2,   120] loss: 0.001                                                          
[2,   140] loss: 0.001                                                          
[2,   160] loss: 0.001                                                          
[2,   180] loss: 0.001                                                          
[2,   200] loss: 0.001                                                          
[2,   220] loss: 0.001                                                          
[2,   240] loss: 0.001                                                          
[2,   260] loss: 0.001                                                          
[2,   280] loss: 0.001                                                          
[2,   300] loss: 0.001                                                          
[2,   320] loss: 0.001      

[1,   260] loss: 0.001                                                          
[1,   280] loss: 0.001                                                          
[1,   300] loss: 0.001                                                          
[1,   320] loss: 0.001                                                          
[1,   340] loss: 0.001                                                          
[1,   360] loss: 0.001                                                          
[1,   380] loss: 0.001                                                          
[1,   400] loss: 0.001                                                          
[2,    20] loss: 0.001                                                          
[2,    40] loss: 0.001                                                          
[2,    60] loss: 0.001                                                          
[2,    80] loss: 0.001                                                          
[2,   100] loss: 0.001      

...

[2,   160] loss: 0.000                                                          
[2,   180] loss: 0.000                                                          
[2,   200] loss: 0.000                                                          
[2,   220] loss: 0.000                                                          
[2,   240] loss: 0.000                                                          
[2,   260] loss: 0.000                                                          
[2,   280] loss: 0.000                                                          
[2,   300] loss: 0.000                                                          
[2,   320] loss: 0.000                                                          
[2,   340] loss: 0.000                                                          
[2,   360] loss: 0.000                                                          
[2,   380] loss: 0.000                                                          
[2,   400] loss: 0.000      

[1,   340] loss: 0.000                                                          
[1,   360] loss: 0.000                                                          
[1,   380] loss: 0.000                                                          
[1,   400] loss: 0.000                                                          
[2,    20] loss: 0.000                                                          
[2,    40] loss: 0.000                                                          
[2,    60] loss: 0.000                                                          
[2,    80] loss: 0.000                                                          
[2,   100] loss: 0.000                                                          
[2,   120] loss: 0.000                                                          
[2,   140] loss: 0.000                                                          
[2,   160] loss: 0.000                                                          
[2,   180] loss: 0.000      

[1,   120] loss: 0.000                                                          
[1,   140] loss: 0.000                                                          
[1,   160] loss: 0.000                                                          
[1,   180] loss: 0.000                                                          
[1,   200] loss: 0.000                                                          
[1,   220] loss: 0.000                                                          
[1,   240] loss: 0.000                                                          
[1,   260] loss: 0.000                                                          
[1,   280] loss: 0.000                                                          
[1,   300] loss: 0.000                                                          
[1,   320] loss: 0.000                                                          
[1,   340] loss: 0.000                                                          
[1,   360] loss: 0.000      

Accuracy of lisa_simpson : 52 %                                                 
Accuracy of marge_simpson : 64 %                                                
Accuracy of milhouse_van_houten : 65 %                                          
Accuracy of moe_szyslak : 62 %                                                  
Accuracy of ned_flanders : 52 %                                                 
Accuracy of principal_skinner : 58 %                                            
[1,    20] loss: 0.000                                                          
[1,    40] loss: 0.000                                                          
[1,    60] loss: 0.000                                                          
[1,    80] loss: 0.000                                                          
[1,   100] loss: 0.000                                                          
[1,   120] loss: 0.000                                                          
[1,   140] loss: 0.000      

[2,   400] loss: 0.000                                                          
training time                                                                   
5.300448894500732                                                               
Finished Training                                                               
runs of batches                                                                 
100                                                                             
Accuracy of the network on the 10000 test images: 55 %                          
Accuracy of bart_simpson : 56 %                                                 
Accuracy of charles_montgomery_burns : 70 %                                     
Accuracy of homer_simpson : 48 %                                                
Accuracy of krusty_the_clown : 47 %                                             
Accuracy of lisa_simpson : 52 %                                                 
Accuracy of marge_simpson : 